In [14]:
import pandas as pd
import os
import pickle

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
from scripts.backtesting.backtesting import Strategy, Backtest 
from scripts.backtesting.lib import crossover


C:\Users\lbisa\Dropbox\Cripto\python\InvestingEvolution\ie\www


In [15]:
klines_file = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './klines/0m15/Completo_XRPUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
#klines_file = './klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    data = pickle.load(file)
    data.set_index('datetime',inplace=True)
    data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
data.tail()

,Open,High,Low,Close,Volume
datetime,,,,,
2023-07-28,29222.78,29542.22,29123.12,29314.14,23993.61627
2023-07-29,29314.14,29406.92,29256.18,29352.90,10851.36844
2023-07-30,29352.90,29449.00,29033.24,29281.09,15706.97441
2023-07-31,29281.09,29530.00,29101.80,29232.25,22605.48964
2023-08-01,29232.26,29739.25,28585.70,29705.99,44719.65162


In [16]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

In [17]:


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            order = self.buy(size=0.5)
            #print('usd_to_buy: ',usd_to_buy,' size: ',size,' order: ',order)
            
        
        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            #order = self.sell()


In [18]:
#Quita las primeras 210 velas, que son agregadas al descargar datos para preveer el armado de indicadores 
#y que la estrategia inicie con los indicadores previamente armados 
SmaCross.exclude_first_data = 210



class Bot(Backtest):
    
    def __init__(self,data,strategy):
        #Ajusa el precio a valores entre referenciales porque Backtesting.py limita los decimales en el size para las compras
        self.exclude_first_data = strategy.exclude_first_data
        cash = 100000

        super().__init__(data, strategy, cash=cash, commission=.002)
    
 
        
    
bot = Bot(data,SmaCross)
stats = bot.run()

bot.plot(open_browser=True)

stats


_plotting.py compare len  943 943


Start                     2021-01-01 00:00:00
End                       2023-08-01 00:00:00
Duration                    942 days 00:00:00
Exposure Time [%]                   37.115589
Equity Final [$]                  74506.59578
Equity Peak [$]                  120398.03668
Return [%]                         -25.493404
Buy & Hold Return [%]                1.276094
Return (Ann.) [%]                  -10.765795
Volatility (Ann.) [%]               16.650543
Sharpe Ratio                        -0.646573
Sortino Ratio                          -0.805
Calmar Ratio                        -0.275854
Max. Drawdown [%]                  -39.027192
Avg. Drawdown [%]                   -6.109176
Max. Drawdown Duration      891 days 00:00:00
Avg. Drawdown Duration      130 days 00:00:00
# Trades                                   26
Win Rate [%]                        30.769231
Best Trade [%]                      30.883415
Worst Trade [%]                    -20.861691
Avg. Trade [%]                    

In [19]:

print(f'Periodo: {stats.Start} al {stats.End}')
print(f'Duracion: {stats.Duration}')


stats._trades

Periodo: 2021-01-01 00:00:00 al 2023-08-01 00:00:00
Duracion: 942 days 00:00:00


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,243,269,37010.53332,48440.65,11430.11668,0.308834,2021-02-04,2021-03-02,None,26 days
1,1,277,293,55963.29318,55025.59,-937.70318,-0.016756,2021-03-10,2021-03-26,None,16 days
2,1,301,321,57166.04388,51125.14,-6040.90388,-0.105673,2021-04-03,2021-04-23,None,20 days
3,1,333,343,57550.98222,46762.99,-10787.99222,-0.187451,2021-05-05,2021-05-15,None,10 days
4,1,374,383,40224.32808,34663.09,-5561.23808,-0.138256,2021-06-15,2021-06-24,None,9 days
5,1,395,401,34288.45002,33086.63,-1201.82002,-0.035050,2021-07-06,2021-07-12,None,6 days
6,1,416,463,39536.78574,46025.24,6488.45426,0.164112,2021-07-27,2021-09-12,None,47 days
7,1,486,513,51574.93398,60911.11,9336.17602,0.181022,2021-10-05,2021-11-01,None,27 days
8,1,519,531,63400.13718,58052.24,-5347.89718,-0.084352,2021-11-07,2021-11-19,None,12 days
9,1,568,576,50877.04098,46446.10,-4430.94098,-0.087091,2021-12-26,2022-01-03,None,8 days
